In [ ]:
pip install pandas

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3596288004.py, line 1)

In [6]:
pip install pandas reportlab

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\YOGA\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [14]:
pip install pandas reportlab pillow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\YOGA\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [73]:
import pandas as pd
from pathlib import Path
import os, platform, subprocess
from reportlab.lib.pagesizes import letter
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.pdfbase.ttfonts import TTFont

# ==== File paths ====
csv_path = Path(r"C:\Grow with google\dashboard files\admet_output.csv")
image_path = Path(r"C:\Grow with google\dashboard files\chart_output.png")
output_pdf = csv_path.with_name("structured_admet_report.pdf")

# ==== Load CSV ====
df = pd.read_csv(csv_path)
df.columns = df.columns.str.strip()

# ==== Select compound ====
compound_data = df.iloc[0]  # First compound in the file

def resolve_units(prop):
    return {
        "molecular_weight": "Dalton",
        "logp": "log-ratio",
        "tpsa": "A²",
        "ld50_zhu": "log(1/(mol/kg))",
        "clearance_hepatocyte_az": "µL/min/10^6 cells",
        "clearance_microsome_az": "µL/min/mg",
        "half_life_obach": "hr",
        "ppbr_az": "%",
        "vdss_lombardo": "L/kg",
        "solubility_aqsoldb": "log(mol/L)",
        "lipophilicity_astrazeneca": "log-ratio",
        "caco2_wang": "log(10^(-6)-cm/s)",
        "hydrationfreeenergy_freesolv": "kcal/mol"
    }.get(prop.lower(), "")

# ==== Define sections and properties ====
sections = {
    "Physicochemical": [
        "molecular_weight", "logP", "hydrogen_bond_acceptors", "hydrogen_bond_donors", "Lipinski","QED","stereo_centers","tpsa"
    ],
    "Absorption": [
        "Bioavailability_Ma", "PAMPA_NCATS", "Pgp_Broccatelli", "Caco2_Wang","HydrationFreeEnergy_FreeSolv","Lipophilicity_AstraZeneca","Solubility_AqSolDB"
    ],
    "Distribution": [
        "BBB_Martins", "PPBR_AZ", "VDss_Lombardo"
    ],
    "Metabolism": [
        "CYP1A2_Veith", "CYP2C19_Veith", "CYP2C9_Substrate_CarbonMangels", "CYP2C9_Veith","CYP2D6_Substrate_CarbonMangels",
        "CYP2D6_Veith","CYP3A4_Substrate_CarbonMangels","CYP3A4_Veith"
    ],
    "Excretion": [
        "Clearance_Hepatocyte_AZ","Clearance_Microsome_AZ","Half_Life_Obach"
    ],
    "Toxicity": [
        "AMES", "QED","Carcinogens_Lagunin","ClinTox","DILI","NR-AR-LBD","NR-AR","NR-AhR","NR-Aromatase","NR-ER-LBD","NR-ER","NR-PPAR-gamma",
        "SR-ARE","SR-ATAD5","SR-HSE","SR-MMP","SR-p53","Skin_Reaction","hERG","LD50_Zhu"
    ]
}

# === Style Setup ===
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name="SectionHeader", fontSize=11, leading=11, textColor=colors.whitesmoke, backColor=colors.purple, alignment=1, spaceAfter=3, spaceBefore=6))
styles.add(ParagraphStyle(name="TableBody", fontSize=11, leading=11))

# ==== Generate section table ====
def make_section(title, props):
    data = [["Property", "Prediction", "DrugBank Percentile", "Units"]]
    for prop in props:
        prediction = compound_data.get(prop, "N/A")
        percentile_col = f"{prop}_drugbank_approved_percentile"
        percentile = compound_data.get(percentile_col, "N/A")
        units = resolve_units(prop)
        data.append([prop, prediction, percentile, units])
    colWidths = [2.5*inch, 1.5*inch, 1.5*inch, 1.5*inch]
    table = Table(data, colWidths=colWidths, repeatRows=1)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#631B9E")),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('INNERGRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOX',(0,0),(-1,-1),1,colors.grey)
    ]))
    return Paragraph(title, getSampleStyleSheet()['Heading2']), table

# ==== Build PDF ====
doc = SimpleDocTemplate(str(output_pdf), pagesize=letter)
styles = getSampleStyleSheet()
elements = [
    Paragraph("ADMET Properties Report", styles['Title']),
    Spacer(1, 6)
]

# Insert image if it exists
if image_path.exists():
    elements += [
        Paragraph("Radar Chart Overview", styles['Heading2']),
        Spacer(1, 3),
        Image(str(image_path), width=5*inch, height=5*inch),
        Spacer(1, 6)
    ]

# Add all sections
for section_title, properties in sections.items():
    header, table = make_section(section_title, properties)
    elements.extend([header, Spacer(1, 3), table, Spacer(1, 6)])

# Generate PDF
doc.build(elements)
print(f"PDF generated: {output_pdf}")

# ==== Automatically open PDF ====
def open_file(filepath):
    if platform.system() == "Windows":
        os.startfile(filepath)
    elif platform.system() == "Darwin":  # macOS
        subprocess.run(["open", filepath])
    else:  # Linux and others
        subprocess.run(["xdg-open", filepath])

open_file(str(output_pdf))


PDF generated: C:\Grow with google\dashboard files\structured_admet_report.pdf
